# Batch Testing Scripts
To test all of the variations of our models, we'll submit batch jobs to the clusters (using `sbatch`) to test *everything, everywhere, all at once*.

In [ ]:
"""
A simple loop to generate dead Simple Queue (dSQ) files for batch training
Usage: change the values as you desire, then run the script to generate a txt file each line of which is a job.
Then log into the cluster and execute two commands:
module load dSQ
dSQ --job-file {specified name}.txt
"""
import numpy as np
import math
# the name of your testing job, for easy reference.
name = 'AUNT_HILLARY'
datasets_to_use = [ # comment out any you don't want 
    "circle2d",
    "circle3d", # The dataset chosen will also specify the sigma graph and flow_strength graph
    "swiss roll",
    "branch",
    "clusters"
]
# Training specifications
ts = [
    '1',
    '1 2 4',
    '1 2 4 8',
]
sigma_embedding = [
    0.5,
    # 1,
    # 5,
    # 15,
]
flow_strength_embedding = [
    1,
    # 2,
    # 4,
]
learning_rate = [
    # 1e-2,
    1e-3,
    # 1e-4
]
flow_artist = [
    'ReLU',
    'Gaussian'
]
flow_artist_shape = [
    '2 4 8 4 2',
    # '2 8 4 2',
    # '2 4 8 16 16 8 4 2',
    # '2 4 2'
]
num_flow_gaussians = [
    25,
    # 36,
    # 64,
]
embedder = [
    'feedforward',
    'learnable diffusion map',
    # 'fixed diffusion map',
    # 'random initialization'
]
decoder = [
    'feedforward',
    'None'
]
loss_weights = [
    '1 0 0 0',
    '0 0 0 1',
    '0.8 0.1 0.1 0',
    # '0.7 0.2 0.1 0',
    # '0.6 0.2 0.2 0',
    # '0.5 0.2 0.3 0',
    '0.5 0 0 0.5',
    '0 0 0 1',
]

num_epochs = 5000
model_args = {}
use_cuda = True

file = open(name+'.txt','w')
for set in datasets_to_use:
    for t in ts:
        for sigma in sigma_embedding:
            for flow_strength in flow_strength_embedding:
                for lr in learning_rate:
                    for emb in embedder:
                        for dec in decoder:
                            for loss_w in loss_weights:           
                                for fa in flow_artist:
                                    if fa == 'ReLU':
                                        for fa_shape in flow_artist_shape:
                                            command = f"module load miniconda; conda activate flowembed; python argrun.py --dataset '{set}' --t '{t}' --flow_strenth '{flow_strength}' --lr '{lr}' --embedder '{emb}' --decoder '{dec}' --loss_weights '{loss_w}' --flow_artist '{fa}' --fa_shape '{fa_shape}'"
                                            command += '\n'
                                            file.write(command)
                                    else:
                                        for n_gauss in num_flow_gaussians:
                                            command = f"module load miniconda; conda activate flowembed; python argrun.py --dataset '{set}' --t '{t}' --flow_strenth '{flow_strength}' --lr '{lr}' --embedder '{emb}' --decoder '{dec}' --loss_weights '{loss_w}' --flow_artist '{fa}' --num_gaussians '{n_gauss}'"
                                            command += '\n'
                                            file.write(command)
file.close()
             